In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.layers import Flatten, Dense, Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
import numpy as np# Visualize data

# Visualize data

In [ ]:
pip install glob2

In [ ]:
pip install imutils

In [ ]:
import pandas as pd
import glob2
import os
import matplotlib.pyplot as plt
from imutils import paths

In [ ]:
classNames = ['aug_handwritten','aug_printed']

In [ ]:
# path = '/kaggle/input/data-train/TrainCompeticion_WITHGroundTruth/'

In [ ]:
class_1 = list(paths.list_images('/kaggle/input/agu-handwritten/aug_handwritten'))
print("len class 1: ", len(class_1))
class_1_labels = ['1']*len(class_1)

In [ ]:
class_2 = list(paths.list_images('/kaggle/input/aug-printed/aug_printed'))
print("len class 2: ", len(class_2))
class_2_labels = ['2']*len(class_2)

In [ ]:
labels = class_1_labels + class_2_labels 
image_links = class_1 + class_2
data = pd.DataFrame({'labels': labels, 'image_links':image_links})
data.groupby(labels).image_links.count().plot.bar()
plt.title('Number of images in each class in printed')
plt.show()

# Split data

In [ ]:
from sklearn.model_selection import train_test_split
images_train, images_val, y_label_train, y_label_val = train_test_split(image_links, labels, random_state=42, stratify = labels)

print('images_train len: {}, image_test shape: {}'.format(len(images_train), len(images_val)))

# Augmentated import numpy as np

In [ ]:
import numpy as np
from tensorflow.keras.utils import Sequence, to_categorical
import cv2

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self,
                 all_filenames, 
                 labels, 
                 batch_size, 
                 index2class,
                 input_dim,
                 n_channels,
                 n_classes=2, 
                 normalize=True,
                 zoom_range=[0.8, 1],
                 rotation=15,
                 brightness_range=[0.8, 1],
                 shuffle=True):
        
        self.all_filenames = all_filenames
        self.labels = labels
        self.batch_size = batch_size
        self.index2class = index2class
        self.input_dim = input_dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.normalize = normalize
        self.zoom_range = zoom_range
        self.rotation = rotation
        self.brightness_range = brightness_range
        self.on_epoch_end()

    def __len__(self):
       
        return int(np.floor(len(self.all_filenames) / self.batch_size))

    def __getitem__(self, index):
        
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        all_filenames_temp = [self.all_filenames[k] for k in indexes]
        X, y = self.__data_generation(all_filenames_temp)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.all_filenames))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, all_filenames_temp):
        X = np.empty((self.batch_size, *self.input_dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        for i, fn in enumerate(all_filenames_temp):
            img = cv2.imread(fn)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.input_dim)
            img_reshape = img.reshape(-1, 3)
            
            if self.normalize:
              mean = np.mean(img_reshape, axis=0)
              std = np.std(img_reshape, axis=0)
              img = (img-mean)/std

            if self.zoom_range:
              zoom_scale = 1/np.random.uniform(self.zoom_range[0], self.zoom_range[1])
              (h, w, c) = img.shape
              img = cv2.resize(img, (int(h*zoom_scale), int(w*zoom_scale)), interpolation = cv2.INTER_LINEAR)
              (h_rz, w_rz, c) = img.shape
              start_w = np.random.randint(0, w_rz-w) if (w_rz-w) > 0 else 0
              start_h = np.random.randint(0, h_rz-h) if (h_rz-h) > 0 else 0
              # print(start_w, start_h)
              img = img[start_h:(start_h+h), start_w:(start_w+w), :].copy()
            
            if self.rotation:
              (h, w, c) = img.shape
              angle = np.random.uniform(-self.rotation, self.rotation)
              RotMat = cv2.getRotationMatrix2D(center = (w, h), angle=angle, scale=1)
              img = cv2.warpAffine(img, RotMat, (w, h))

            if self.brightness_range:
              scale_bright = np.random.uniform(self.brightness_range[0], self.brightness_range[1])
              img = img*scale_bright
            
            # dataset_task2/data_classify/0/*.jpg
            label = fn.split("/")[-3]
            label = self.index2class[label]
            # print("label: ", label)
    
            X[i,] = img

            # Lưu class
            y[i] = label
        # lb = LabelBinarizer()
        # y = lb.fit_transform(y)
        return X, y

In [ ]:
dict_labels = {
    'aug_handwritten': 0,
    'aug_printed': 1
}

In [ ]:
train_generator = DataGenerator(
    all_filenames = images_train,
    labels = y_label_train,
    batch_size = 16,
    index2class = dict_labels,
    input_dim = (224, 224),
    n_channels = 3,
    n_classes = 2,
    normalize = False,
    zoom_range = [0.5, 1],
    rotation = False,
    brightness_range=[0.8, 1],
    shuffle = True
)

In [ ]:
val_generator = DataGenerator(
    all_filenames = images_val,
    labels = y_label_val,
    batch_size = 16,
    index2class = dict_labels,
    input_dim = (224, 224),
    n_channels = 3,
    n_classes = 2,
    normalize = False,
    zoom_range = [0.5, 1],
    rotation = False,
    brightness_range =[0.8, 1],
    shuffle = False
)

# Check augment

In [ ]:
check_aug=['/kaggle/input/agu-handwritten/aug_handwritten/Arabic/image_0_1000.png']*32

check_generator = DataGenerator(
    all_filenames = images_val,
    labels = y_label_val,
    batch_size = 20,
    index2class = dict_labels,
    input_dim = (224, 224),
    n_channels = 3,
    n_classes = 2,
    normalize = False,
    zoom_range = [0.5, 1],
    rotation = 15,
    brightness_range = [0.5, 1.5],
    shuffle = False
)

In [ ]:
X_batch, y_batch = check_generator.__getitem__(0)

print(X_batch.shape)
print(y_batch.shape)

In [ ]:
import matplotlib.pyplot as plt

fg, ax = plt.subplots(4, 5, figsize=(20, 16))
fg.suptitle('Augumentation Images')

for i in np.arange(4):
  for j in np.arange(5):
    ax[i, j].imshow(X_batch[i + j + j*i]/255.0)
    ax[i, j].set_xlabel('Image '+str(i+j+j*i))
    ax[i, j].axis('off')
plt.show()

# Fine-tune

In [ ]:
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB7
from keras.layers import Activation,Flatten, Dense, Input, Dropout, AveragePooling2D

In [ ]:
def create_model(baseModel, number_class, lr=1e-4, decay=1e-4/25):

    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(3, 3))(headModel)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(1024, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(number_class, activation="softmax")(headModel)
    model = Model(inputs=baseModel.input, outputs=headModel)

    for layer in baseModel.layers:
      layer.trainable = False
    
    # compile model
    optimizer = Adam(lr=lr, decay = decay)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,metrics="accuracy")

    return model

In [ ]:
baseModel = EfficientNetB7(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
model = create_model(baseModel, 2, lr=1e-4, decay=1e-4/25)

# Train step 1

In [ ]:
history = model.fit(train_generator,
                   validation_data= val_generator, 
                   epochs=5) 

# Train step 2

In [ ]:
from keras.optimizers import SGD

In [ ]:
for layer in baseModel.layers[20:]:
    layer.trainable = True

#recompile the model
print("[INFO] re-compiling model...")
opt = SGD(lr=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
            metrics=["accuracy"])

In [ ]:
mkdir /kaggle/working/model

In [ ]:
mkdir /kaggle/working/checkpoint

In [ ]:
my_checkpointer = [
                EarlyStopping(monitor='val_loss', patience=5, verbose=0),
                ModelCheckpoint(filepath="/kaggle/working/checkpoint/effb7_10epoch.h5", verbose=2, save_weights_only=True)
                ]

In [ ]:
history = model.fit(train_generator,
                  #  steps_per_epoch=20, 
                   validation_data= val_generator, 
                  #  validation_steps=10, 
                   epochs=5,
                   callbacks=my_checkpointer)

In [ ]:
model.save('/kaggle/working/model/model_hand_printed.h5')


In [ ]:
cd /kaggle/working

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink(r'model/model_hand_printed.h5')